In [4]:
import os
import torch
import copy
import tensorflow as tf
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
# from Downloads import LSTMSudokuClassifier as LSTMC
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from random import randint

from tqdm import tqdm_notebook
tqdm_notebook.monitor_interval = 0

In [5]:
class LSTMClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, n_label, batch_size, n_layers, use_gpu):
        super(LSTMClassifier, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.use_gpu = use_gpu
        self.n_layers = n_layers
               
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, n_layers, bidirectional=True)
        self.lstm2 = nn.LSTM(input_dim, hidden_dim, n_layers, bidirectional=True)
        self.lstm3 = nn.LSTM(input_dim, hidden_dim, n_layers, bidirectional=True)
        
        self.hidden2label = nn.Sequential(torch.nn.Linear(2*hidden_dim, n_label),
                                          torch.nn.Softmax(dim = -1))
        self.r_hidden = self.init_hidden()
        self.c_hidden = self.init_hidden()
        self.s_hidden = self.init_hidden()

    def init_hidden(self):
        if self.use_gpu:
            h0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim))
        return (h0, c0)
    
    def ManipulateSquare(self, lstm_out_square):
        counter = 0
        inner_stack = None
        outer_stack = None
        for i in range(3):
            for j in range(3):
                x = lstm_out_square[counter,:,:]
                counter += 1
                x = x.view(3,3,2*hidden_dim)#change to 2*hidden_dim
                if inner_stack is None:
                    inner_stack = x
                else:
                    inner_stack = torch.cat((inner_stack,x),1)        
            if outer_stack is None:
#                 print(inner_stack.shape)
                outer_stack = inner_stack
                inner_stack = None
#                 print('B = ',outer_stack)
            else:
#                 print(inner_stack.shape)
                outer_stack = torch.cat((outer_stack,inner_stack),0)
#                 print('B = ',outer_stack)
                inner_stack = None
        return outer_stack

    def forward(self, row_tensor, col_tensor, square_tensor):
        
        lstm_out_row, hidden_row = self.lstm1(row_tensor, self.r_hidden)
        lstm_out_col, hidden_col = self.lstm2(col_tensor, self.c_hidden)
        lstm_out_square, hidden_square = self.lstm3(square_tensor, self.s_hidden)
        
#         self.r_hidden = hidden_row
#         self.c_hidden = hidden_col
#         self.s_hidden = hidden_square
        
        # Do manipulation Here
        #Row & Col Manipulation
        row_col = lstm_out_row + lstm_out_col +lstm_out_square 
        #Square Manipulation
        lstm_out = row_col 
        lstm_out_tensor = lstm_out
#         if self.use_gpu:
#             lstm_out_tensor = lstm_out.cuda()
#         print("final tensor", lstm_out_tensor.shape, lstm_out_tensor)
        
        #Sum Square Row Col
        y  = self.hidden2label(lstm_out_tensor)
#         print('Output Shape ', y.shape)
        return y

In [6]:
class InputGrid():
    
    def __init__(self, Grid):
        self.Grid = Grid
        self.gridLength = len(Grid[0])
#         self.labels = labels
        
    def getlims(self, i):
        if 0 <= i <= 2:
            rowlims = [0,3]
        elif 3 <= i <= 5:
            rowlims = [3,6]
        elif 6 <= i <= 8:
                rowlims = [6,9]
        return rowlims

    def get_square(self, i):
        grid_square = None
        if i < 3:
            if i%3 == 0:
                grid_square = self.Grid[0:3,0:3]
            elif i%3 == 1:
                grid_square = self.Grid[0:3,3:6]
            elif i%3 == 2:
                grid_square = self.Grid[0:3,6:9]
        elif i < 6:
            if i%3 == 0:
                grid_square = self.Grid[3:6,0:3]
            elif i%3 == 1:
                grid_square = self.Grid[3:6,3:6]
            elif i%3 == 2:
                grid_square = self.Grid[3:6,6:9]
        elif i < 9:
            if i%3 == 0:
                grid_square = self.Grid[6:9,0:3]
            elif i%3 == 1:
                grid_square = self.Grid[6:9,3:6]
            elif i%3 == 2:
                grid_square = self.Grid[6:9,6:9]
        return grid_square.flatten()
                
    
    # takes 1D returns 2D
    def one_hot(self, vec):
        one_hot_matrix = []
        for val in vec:
            hot_vec = [0 for _ in range(9)]
            if val > 0:
                hot_vec[int(val)-1] = 1
            one_hot_matrix.append(hot_vec)
        real_one_hot = np.array(one_hot_matrix)
        return real_one_hot
    
    #takes 2D returns 3D
    def getInput(self):
#         print(self.Grid)
        Rows = np.zeros((9,9,9))
        Columns = np.zeros((9,9,9))
        Squares = np.zeros((9,9,9))
        Labels = np.zeros((9,9,9))
        for i in range(self.gridLength):
            hot_row = self.one_hot(self.Grid[i,:])
            hot_column = self.one_hot(self.Grid[:,i])
            hot_square = self.one_hot(self.get_square(i))
#             hot_label = self.one_hot(self.labels[i,:])
            
            Rows[i] = hot_row
            Columns[i] = hot_column
            Squares[i] = hot_square
#             Labels[i] = hot_label
        
        Rows = np.reshape (Rows , (1, 81, 9))
        Columns = np.reshape (Columns, (1, 81, 9))
        Squares = np.reshape (Squares, (1, 81, 9))
        Rows = np.transpose (Rows, ((1, 0, 2)))
        Columns = np.transpose (Columns, ((1, 0, 2)))
        Squares = np.transpose (Squares, ((1, 0, 2)))

        row_tensor = Variable(torch.FloatTensor(Rows))
        col_tensor = Variable(torch.FloatTensor(Columns))
        square_tensor = Variable(torch.FloatTensor(Squares))
#         label_tensor = Variable(torch.LongTensor(Labels))
#         print(row_tensor)
#         print(col_tensor)
#         print(square_tensor)
#         print(label_tensor)
                 
        return row_tensor, col_tensor, square_tensor

In [21]:
# use_plot = True
# use_save = True
# if use_save:
#     import pickle
#     from datetime import datetime

DATA_DIR = 'data'
TRAIN_FILE = 'kaggle_sudoku.csv'
TEST_FILE = 'test.csv'
TRAIN_LABEL = 'train_label.txt'
TEST_LABEL = 'test_label.txt'
save_filepath = 'rnn.pt'

In [25]:
class SudokuDataset(Dataset):
    def __init__(self, fpath):
        
        print(fpath)
        lines = open(fpath, 'r').read().splitlines()[1:]
        nsamples = len(lines)

        X = np.zeros((nsamples, 9*9), np.float32)  
        Y = np.zeros((nsamples, 9*9), np.int32) 

        for i, line in enumerate(lines):
            quiz, solution = line.split(",")
            for j, (q, s) in enumerate(zip(quiz, solution)):
                X[i, j], Y[i, j] = q, s
            if i>=nsamples-1:
                break

        X = np.reshape(X, (-1, 9, 9))
        Y = np.reshape(Y, (-1, 9, 9))
        Y -= 1
        self.X = X
        self.Y = Y
#         print(Y)
#         print(Y.shape)

    def __getitem__(self, index):
        quiz = self.X[index]
        sol = self.Y[index]
        return quiz, sol
    
    def __len__(self):
        return len(self.X)

In [6]:
if __name__=='__main__':
    
    ## parameter setting
    epochs = 100
    batch_size = 1
    use_gpu = torch.cuda.is_available()
    learning_rate = 0.001

    input_dim = 9
    hidden_dim = 200
    n_label = 9
    n_layers = 1
    
    train_path = os.path.join(DATA_DIR, TRAIN_FILE)
    test_path = os.path.join(DATA_DIR, TEST_FILE)
    

     ### ********************create model**************************
    model = LSTMClassifier(input_dim, hidden_dim, n_label, batch_size, n_layers, use_gpu)
    if use_gpu:
        model = model.cuda()
    
    training_set = SudokuDataset(train_path)
    train_loader = DataLoader(training_set,
                          batch_size,
                          shuffle=True,
                          num_workers=0
                          )
    
    print('TRAINING')

    optimizer = optim.Adam(model.parameters())
    loss_function = nn.CrossEntropyLoss()
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    
    
    def adjust_learning_rate(optimizer, epoch):
        lr = learning_rate * (0.1 ** (epoch // 10))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        return optimizer
    
    import pdb
    nsamples = 1000//batch_size

### training procedure

#     t = trange(100, desc='Bar desc', leave=True)
    for epoch in range(epochs):
#         optimizer = adjust_learning_rate(optimizer, epoch)
        total_acc = 0.0
        total_loss = 0.0
        total = 0.0

#         model.hidden = model.init_hidden()

#         t = trange(100000, desc='RNN', leave=True)
        for i, traindata in zip(tqdm_notebook(range(nsamples)), train_loader):
#             traindata = train_loader[i]
            model.zero_grad()
            train_inputs_raw, train_labels_raw = traindata
#             print(train_inputs_raw.shape)

            train_data_numpy = Variable(train_inputs_raw).data.numpy()
            train_labels_numpy = Variable(train_labels_raw).data.numpy()
            train_labels = Variable(torch.LongTensor(train_labels_numpy[0]))

#             pdb.set_trace ()
            input_grid = InputGrid(train_data_numpy[0])
            row_tensor, col_tensor, square_tensor = input_grid.getInput()

            if use_gpu:
                row_tensor, col_tensor, square_tensor, train_labels = \
                        row_tensor.cuda(), col_tensor.cuda(), square_tensor.cuda(), train_labels.cuda()

            model.hidden = model.init_hidden()
            model.batch_size =  batch_size 
            output = model(row_tensor, col_tensor, square_tensor)
            output = output.view(81,9)


#             for i in range(output.size()[1]):
            loss = loss_function(output, train_labels.view(81,))
            loss.backward()
            optimizer.step()

            # calc training acc
            _, predicted = torch.max(output.data, 1)
#             print(predicted.shape)
#             print('TrainPred =  ',predicted, 'While TrainLabel = ' ,train_labels.data)
            acc = (predicted == train_labels.data.view(81)).sum()/81
#             print(acc)
            total += 81
#             print(loss.data[0])
#             print (acc)

#        print("loss:", loss.data[0], "acc", total_acc)
            train_loss_.append(loss.data[0])
            train_acc_.append(acc)

#             t.set_description("Bar desc (file %i)" % iter)
#             tqdm.refresh(self=) # t/o show immediately the update

        print('[Epoch: %3d/%3d] Training Loss: %.3f, Training Acc: %.3f' 
                  % (epoch, epochs, np.mean(train_loss_), np.mean (train_acc_)))
    torch.save(model.state_dict(), save_filepath)


data\kaggle_sudoku.csv
TRAINING


A Jupyter Widget


[Epoch:   0/100] Training Loss: 1.871, Training Acc: 0.492


A Jupyter Widget


[Epoch:   1/100] Training Loss: 1.854, Training Acc: 0.505


...


[Epoch:  97/100] Training Loss: 1.581, Training Acc: 0.792


A Jupyter Widget


[Epoch:  98/100] Training Loss: 1.580, Training Acc: 0.793


A Jupyter Widget


[Epoch:  99/100] Training Loss: 1.579, Training Acc: 0.794


In [33]:
### TEST
epochs = 100
batch_size = 1
use_gpu = torch.cuda.is_available()
learning_rate = 0.001

input_dim = 9
hidden_dim = 200
n_label = 9
n_layers = 1

train_path = os.path.join(DATA_DIR, TRAIN_FILE)
test_path = os.path.join(DATA_DIR, TEST_FILE)


 ### ********************create model**************************
model = LSTMClassifier(input_dim, hidden_dim, n_label, batch_size, n_layers, use_gpu)
if use_gpu:
    model = model.cuda()    


model.load_state_dict(torch.load(save_filepath))
model.eval()
total_acc = 0.0
#     total_loss = 0.0
total = 0.0

test_set = SudokuDataset(train_path)
test_loader = DataLoader(test_set,
                        batch_size,
                        shuffle=True,
                        num_workers=0
                        )
nsamples = 30
test_loss_ = []
test_acc_ = []

for i, testdata in zip(tqdm_notebook(range(nsamples)), test_loader):
    model.zero_grad()
    train_inputs_raw, train_labels_raw = testdata
#             print(train_inputs_raw.shape)

    train_data_numpy = Variable(train_inputs_raw).data.numpy()
    train_labels_numpy = Variable(train_labels_raw).data.numpy()
    train_labels = Variable(torch.LongTensor(train_labels_numpy[0]))

    input_grid = InputGrid(train_data_numpy[0])
    row_tensor, col_tensor, square_tensor = input_grid.getInput()

    if use_gpu:
        row_tensor, col_tensor, square_tensor, train_labels = \
                row_tensor.cuda(), col_tensor.cuda(), square_tensor.cuda(), train_labels.cuda()

    model.hidden = model.init_hidden()
    model.batch_size = batch_size 
    output = model(row_tensor, col_tensor, square_tensor)
    output = output.view(81,9)

#             for i in range(output.size()[1]):
#         loss = loss_function(output, train_labels.view(81,))
# #         loss.backward()
# #         optimizer.step()

    # calc training acc
    _, predicted = torch.max(output.data, 1)
#             print(predicted.shape)
    print('TrainPred = ', predicted.view(9,9), 'While TrainLabel = ' ,train_labels.data.view(9,9))
    acc = (predicted == train_labels.data.view(81)).sum()/81
#             print(acc)
#             print(loss.data[0])
#             print (acc)

#        print("loss:", loss.data[0], "acc", total_acc)
#     test_loss_.append(loss.data[0])
    test_acc_.append(acc)

#             t.set_description("Bar desc (file %i)" % iter)
#             tqdm.refresh(self=) # t/o show immediately the update

print('[Epoch: %3d/%3d], Testing Acc: %.3f' % (1, 1, np.mean (test_acc_)))

data\test.csv


A Jupyter Widget

TrainPred =  
    8     5     8     8     2     7     3     4     0
    2     3     6     8     5     8     7     5     3
    7     3     4     1     2     4     8     3     6
    0     8     5     7     4     7     1     5     5
    2     8     3     2     6     4     0     7     1
    1     1     0     3     1     5     8     7     2
    5     6     6     6     4     1     3     0     8
    3     4     2     7     7     0     6     0     1
    7     8     6     7     3     1     7     2     5
[torch.cuda.LongTensor of size 9x9 (GPU 0)]
 While TrainLabel =  
    3     5     8     6     7     2     1     4     0
    2     1     6     5     4     0     7     8     3
    7     0     4     1     8     3     2     5     6
    0     2     5     8     1     7     3     6     4
    8     7     3     2     6     4     0     1     5
    6     4     1     3     0     5     8     7     2
    5     6     7     0     2     1     4     3     8
    4     3     2     7     5     8     6     0     1
  

 While TrainLabel =  
    8     3     2     4     7     5     1     0     6
    6     7     5     1     0     3     8     2     4
    1     0     4     8     6     2     3     5     7
    3     5     6     2     4     0     7     8     1
    2     1     8     3     5     7     6     4     0
    7     4     0     6     1     8     5     3     2
    4     2     7     5     8     6     0     1     3
    0     8     1     7     3     4     2     6     5
    5     6     3     0     2     1     4     7     8
[torch.cuda.LongTensor of size 9x9 (GPU 0)]

TrainPred =  
    5     8     0     4     4     3     2     7     8
    7     0     6     1     5     2     5     4     1
    2     5     3     7     8     4     6     5     2
    5     0     0     2     7     8     0     6     2
    1     7     6     3     1     8     4     8     5
    3     0     2     6     4     1     7     6     5
    6     1     8     4     0     2     1     4     6
    2     0     0     0     3     8     1     1     6
 


TrainPred =  
    4     2     3     8     3     0     1     5     7
    8     2     7     2     5     5     4     2     0
    0     5     0     6     3     1     1     8     4
    5     2     4     0     2     3     7     4     6
    2     0     8     4     5     1     7     3     5
    6     3     6     1     6     8     5     1     2
    6     8     7     5     0     3     4     6     7
    1     0     6     8     3     8     5     0     0
    2     3     5     6     5     7     8     5     4
[torch.cuda.LongTensor of size 9x9 (GPU 0)]
 While TrainLabel =  
    4     6     3     8     5     0     1     2     7
    8     0     7     2     1     6     4     3     5
    1     5     2     7     3     4     6     8     0
    5     7     1     0     2     3     8     4     6
    3     2     8     4     6     5     7     0     1
    6     4     0     1     7     8     3     5     2
    7     8     4     5     0     1     2     6     3
    0     1     6     3     4     2     5     7     8
 

 While TrainLabel =  
    8     1     6     3     5     7     0     2     4
    7     2     4     8     6     0     5     1     3
    3     5     0     1     4     2     8     6     7
    0     3     8     2     7     6     4     5     1
    2     4     5     0     8     1     3     7     6
    1     6     7     5     3     4     2     8     0
    4     8     1     6     0     5     7     3     2
    5     0     2     7     1     3     6     4     8
    6     7     3     4     2     8     1     0     5
[torch.cuda.LongTensor of size 9x9 (GPU 0)]

TrainPred =  
    5     1     0     3     8     5     5     7     5
    3     2     2     0     4     8     8     6     1
    5     6     4     8     2     7     3     0     1
    4     8     2     5     0     6     1     4     0
    7     3     0     6     8     7     4     2     6
    1     1     7     4     2     3     8     4     5
    2     1     5     6     0     2     4     7     8
    4     1     0     0     3     2     5     4     6
 



[Epoch:   1/  1], Testing Acc: 0.467
